# Knowledgeable Challenge

Here, we perform the same tasks as in the Beginner Challenge, but for a given LiH ansantz.

In [2]:
#Pytket imports
from pytket import Circuit, Qubit
from pytket.circuit.display import render_circuit_jupyter
from pytket.extensions.qiskit import AerBackend
from pytket.extensions.qiskit import qiskit_to_tk, tk_to_qiskit
from pytket.qasm import circuit_to_qasm, circuit_from_qasm
from pytket.circuit import OpType

#Optimization imports
from pytket.passes import *
from pytket.predicates import CompilationUnit
from pytket.transform import CXConfigType

#For running on IBM Quantum Chips
from qiskit import QuantumCircuit, transpile, Aer, IBMQ, execute
from qiskit.tools.jupyter import *
from qiskit.visualization import *
from ibm_quantum_widgets import *
from qiskit.providers.aer import QasmSimulator
import qiskit.providers.aer.noise as noise

#Load IBM account to ensure access to chips
IBMQ.load_account() # Load account from disk
IBMQ.providers()    # List all available providers
provider = IBMQ.load_account()

#For graphing and basic math
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from math import pi

#Sanity check
backend = AerBackend()
backend.required_predicates

configrc.store_credentials:WARNING:2022-08-21 21:53:38,983: Credentials already present. Set overwrite=True to overwrite.
ibmqfactory.load_account:WARNING:2022-08-21 21:53:42,910: Credentials are already in use. The existing account in the session will be replaced.


[NoSymbolsPredicate,
 GateSetPredicate:{ SX Y ZZPhase RangePredicate X YYPhase Z XXPhase S Sdg T Tdg SXdg CnX Barrier CCX H Rx Unitary1qBox Ry Unitary2qBox Rz U3 U2 U1 TK1 CX CY CZ CU1 CU3 SWAP CSWAP noop Measure Reset PhasedX }]

In [3]:
#Import qasm file
qasmfile = "LiHJordanWignerMapper.qasm"
qc = circuit_from_qasm(qasmfile) #generates circuit
qc.measure_all()

[X q[0]; X q[1]; H q[2]; H q[3]; SX q[5]; X q[6]; X q[7]; H q[8]; H q[9]; SX q[11]; H q[0]; H q[1]; CX q[5], q[4]; H q[6]; H q[7]; CX q[11], q[10]; CX q[4], q[3]; CX q[10], q[9]; Rz(0.5) q[3]; Rz(0.5) q[9]; CX q[4], q[3]; CX q[10], q[9]; H q[3]; CX q[5], q[4]; H q[9]; CX q[11], q[10]; SX q[3]; SXdg q[5]; SX q[9]; SXdg q[11]; H q[5]; H q[11]; CX q[5], q[4]; CX q[11], q[10]; CX q[4], q[3]; CX q[10], q[9]; Rz(3.5) q[3]; Rz(3.5) q[9]; CX q[4], q[3]; CX q[10], q[9]; SXdg q[3]; CX q[5], q[4]; SXdg q[9]; CX q[11], q[10]; H q[5]; H q[11]; SX q[5]; SX q[11]; CX q[5], q[4]; CX q[11], q[10]; CX q[4], q[3]; CX q[10], q[9]; CX q[3], q[2]; CX q[9], q[8]; Rz(0.5) q[2]; Rz(0.5) q[8]; CX q[3], q[2]; CX q[9], q[8]; H q[2]; CX q[4], q[3]; H q[8]; CX q[10], q[9]; SX q[2]; CX q[5], q[4]; SX q[8]; CX q[11], q[10]; SXdg q[5]; SXdg q[11]; H q[5]; H q[11]; CX q[5], q[4]; CX q[11], q[10]; CX q[4], q[3]; CX q[10], q[9]; CX q[3], q[2]; CX q[9], q[8]; Rz(3.5) q[2]; Rz(3.5) q[8]; CX q[3], q[2]; CX q[9], q[8]; SXdg 

In [3]:
#Draw with TKET
render_circuit_jupyter(qc)

In [4]:
print("TKET circuit depth reading: " + str(qc.depth())) 

TKET circuit depth reading: 9835


In [14]:
#Some basic optimization using just passes
seqpass = SequencePass([CommuteThroughMultis(), RemoveRedundancies(), 
                        SimplifyInitial(), KAKDecomposition(), RemoveDiscarded(),
                        ZZPhaseToRz()])
reppass = RepeatPass(seqpass)
cu = CompilationUnit(qc)
reppass.apply(cu)
qc1 = cu.circuit

#Check to see how it affected the circuit depth
print("New TKET circuit depth reading: " + str(qc1.depth())) 

New TKET circuit depth reading: 7809


In [75]:
#Figure out basis gates for more in-depth optimization
backend = provider.get_backend('ibmq_belem')
backend.configuration().basis_gates

['id', 'rz', 'sx', 'x', 'cx', 'reset']

In [76]:
#qc_qiskit = tk_to_qiskit(qc1)
#qc_basis = transpile(qc_qiskit, backend)

We approach a problem here where since the circuit cannot be visualized without Jupyter crashing, it is assumed that every qubit in this circuit is being acted on and not just used to store values. Therefore, we cannot optimize this circuit to reduce the number of qubits. From here, we will use IBM's QASM simulator, but one could optimize their circuit in accordance to the basis gates used by their quantum chip. 

In [77]:
#Set up noise model based on ibmq_belem specs that can be tuned later
readout_error = 2.624 * 10**(-2)
CNOT_error = 1.231 * 10**(-2)

error_1 = noise.depolarizing_error(readout_error, 1) #single gate errors
error_2 = noise.depolarizing_error(readout_error, 2) #two qubit gate errors

noise_model = noise.NoiseModel()
noise_model.add_all_qubit_quantum_error(error_1, ['rz', 'x', 'sx'])
noise_model.add_all_qubit_quantum_error(error_2, ['cx'])

basis_gates = provider.get_backend('ibmq_belem').configuration().basis_gates

#Perform a noise simulation
result = execute(qc_qiskit, Aer.get_backend('qasm_simulator'),
                 basis_gates=basis_gates,
                 noise_model=noise_model).result()

qc_basis = transpile(qc_qiskit, Aer.get_backend('qasm_simulator'))

In [78]:
#Used Qiskit to automatically further optimize the circuit
print("New circuit depth reading: " + str(qc_basis.depth())) 

New circuit depth reading: 7231


From this, it can be concluded that there was some additional optimization that could have been performed, as Qiskit's automatic optimization is typically quite thorough. One possibility is removing orbitals 3 and 4, since those are unoccupied for this molecule. However, performing this task when only given a specific circuit ansatz and not further specifications of the setup of the molecule is a little improbable. It is most likely that Qiskit found ways to optimize the circuit based on `ibmq_belem`'s backend, which we could not do because of the number of qubits in our ansatz. However, these will be kept in mind when completing the advanced challenge.